# Erkennung von Aktivitäten aus Smartphone-Messungen

### Schritt 1: Einlesen der Daten

### Schritt 2: Überprüfen und Bereinigen von doppelten Spaltennamen

In [15]:
import pandas as pd

# Funktion zum Bereinigen und Konvertieren der Daten
def clean_and_convert(data):
    # Ersetzen von Kommas durch Punkte
    data = data.replace(',', '.', regex=True)
    # Konvertieren der Daten in numerische Werte

    numeric_columns = data.columns.drop(['subject'])
    for column in numeric_columns:
        data[column] = pd.to_numeric(data[column], errors='coerce')
    return data

# Daten einlesen
data = pd.read_csv('measures.csv', sep=';')
to_predict = pd.read_csv('to_predict.csv', sep=';')

# Bereinigen und Konvertieren der Daten
data = clean_and_convert(data)
to_predict = clean_and_convert(to_predict)

# Überprüfen der ersten Zeilen der bereinigten Daten
print(data.head())
print(to_predict.head())

# Überprüfen der Spaltennamen
print(data.columns)
print(to_predict.columns)

# Überprüfen der eindeutigen Werte in der 'subject'-Spalte
print(data['subject'].unique())


   tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  tBodyAcc-std()-X  \
0           0.288585          -0.020294          -0.132905         -0.995279   
1           0.278419          -0.016411          -0.123520         -0.998245   
2           0.279653          -0.019467          -0.113462         -0.995380   
3           0.279174          -0.026201          -0.123283         -0.996091   
4           0.276629          -0.016570          -0.115362         -0.998139   

   tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  tBodyAcc-mad()-Y  \
0         -0.983111         -0.913526         -0.995112         -0.983185   
1         -0.975300         -0.960322         -0.998807         -0.974914   
2         -0.967187         -0.978944         -0.996520         -0.963668   
3         -0.983403         -0.990675         -0.997099         -0.982750   
4         -0.980817         -0.990482         -0.998321         -0.979672   

   tBodyAcc-mad()-Z  tBodyAcc-max()-X  ...  fBodyBodyGyr

In [16]:
import pandas as pd

def maybe_dedup_names(names):
    # Hilfsfunktion zur Bereinigung doppelter Spaltennamen
    counts = {}
    result = []
    for name in names:
        if name not in counts:
            counts[name] = 0
        else:
            counts[name] += 1
            name = f"{name}.{counts[name]}"
        result.append(name)
    return result

# Überprüfen auf doppelte Spaltennamen
data_columns = data.columns
to_predict_columns = to_predict.columns

# Bereinigen von doppelten Spaltennamen
data.columns = maybe_dedup_names(data_columns)
to_predict.columns = maybe_dedup_names(to_predict_columns)

# Überprüfen der bereinigten Spaltennamen
print(data.columns)
print(to_predict.columns)


Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'fBodyBodyGyroJerkMag-kurtosis()', 'angle(tBodyAccMean,gravity)',
       'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'activity'],
      dtype='object', length=563)
Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'fBodyBodyGyroJerkMag-skewness()', 'fBodyBodyGyroJerkMag-kurtosis()',
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(

### Schritt 3: Datenaufteilung in Trainings- und Testmengen

In [17]:
# Datenaufteilung
train_data = data[data['subject'].isin([1, 3, 5, 6])]
test_data = data[data['subject'].isin([27, 28, 29, 30])]

# Features und Zielvariable
X_train = train_data.drop(['subject', 'activity'], axis=1)
y_train = train_data['activity']
X_test = test_data.drop(['subject', 'activity'], axis=1)
y_test = test_data['activity']

# Vorhersagedaten
X_to_predict = to_predict.drop(['subject'], axis=1)

# Formen der Datensätze anzeigen
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_to_predict.shape)

(0, 561) (0,) (0, 561) (0,) (2947, 561)


In [18]:
import pandas as pd

# Daten einlesen
data = pd.read_csv('measures.csv', sep=';')
to_predict = pd.read_csv('to_predict.csv', sep=';')

# Überprüfen der ersten Zeilen der Daten
print(data.head())
print(to_predict.head())

# Überprüfen der Spaltennamen
print(data.columns)
print(to_predict.columns)

# Überprüfen der eindeutigen Werte in der 'subject'-Spalte
print(data['subject'].unique())



  tBodyAcc-mean()-X tBodyAcc-mean()-Y tBodyAcc-mean()-Z tBodyAcc-std()-X  \
0        0,28858451      -0,020294171       -0,13290514       -0,9952786   
1        0,27841883      -0,016410568       -0,12352019      -0,99824528   
2        0,27965306      -0,019467156       -0,11346169      -0,99537956   
3        0,27917394      -0,026200646       -0,12328257      -0,99609149   
4        0,27662877      -0,016569655       -0,11536185      -0,99813862   

  tBodyAcc-std()-Y tBodyAcc-std()-Z tBodyAcc-mad()-X tBodyAcc-mad()-Y  \
0      -0,98311061      -0,91352645      -0,99511208      -0,98318457   
1      -0,97530022      -0,96032199      -0,99880719      -0,97491437   
2      -0,96718701      -0,97894396      -0,99651994      -0,96366837   
3       -0,9834027       -0,9906751      -0,99709947      -0,98274984   
4      -0,98081727      -0,99048163      -0,99832113      -0,97967187   

  tBodyAcc-mad()-Z tBodyAcc-max()-X  ... fBodyBodyGyroJerkMag-kurtosis()  \
0      -0,92352702      -0,9

### Schritt 4: Training des SVM-Modells

In [19]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# SVM Modelltraining
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Modellbewertung
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy:.2f}')

ValueError: Found array with 0 sample(s) (shape=(0, 561)) while a minimum of 1 is required by SVC.

### Schritt 5: Vorhersagen auf unbekannten Daten

In [ ]:
# Vorhersagen auf unbekannten Daten
predictions = svm_model.predict(X_to_predict)

# Vorhersagen speichern
to_predict['activity'] = predictions
to_predict.to_csv('predictions.csv', index=False)